# Inference

## Imports & Args

In [1]:
import argparse
import json
import logging
import os
import random
from io import open
import numpy as np
import math
import _pickle as cPickle
from scipy.stats import spearmanr

from tensorboardX import SummaryWriter
from tqdm import tqdm
from bisect import bisect
import yaml
from easydict import EasyDict as edict
import sys
import pdb

import torch
import torch.nn.functional as F
import torch.nn as nn

from vilbert.task_utils import (
    LoadDatasetEval,
    LoadLosses,
    ForwardModelsTrain,
    ForwardModelsVal,
    EvaluatingModel,
)

import vilbert.utils as utils
import torch.distributed as dist

In [2]:
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger = logging.getLogger(__name__)

In [3]:
parser = argparse.ArgumentParser()

parser.add_argument(
    "--bert_model",
    default="bert-base-uncased",
    type=str,
    help="Bert pre-trained model selected in the list: bert-base-uncased, "
    "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.",
)
parser.add_argument(
    "--from_pretrained",
    default="bert-base-uncased",
    type=str,
    help="Bert pre-trained model selected in the list: bert-base-uncased, "
    "bert-large-uncased, bert-base-cased, bert-base-multilingual, bert-base-chinese.",
)
parser.add_argument(
    "--output_dir",
    default="results",
    type=str,
    help="The output directory where the model checkpoints will be written.",
)
parser.add_argument(
    "--config_file",
    default="config/bert_config.json",
    type=str,
    help="The config file which specified the model details.",
)
parser.add_argument(
    "--no_cuda", action="store_true", help="Whether not to use CUDA when available"
)
parser.add_argument(
    "--do_lower_case",
    default=True,
    type=bool,
    help="Whether to lower case the input text. True for uncased models, False for cased models.",
)
parser.add_argument(
    "--local_rank",
    type=int,
    default=-1,
    help="local_rank for distributed training on gpus",
)
parser.add_argument(
    "--seed", type=int, default=42, help="random seed for initialization"
)
parser.add_argument(
    "--fp16",
    action="store_true",
    help="Whether to use 16-bit float precision instead of 32-bit",
)
parser.add_argument(
    "--loss_scale",
    type=float,
    default=0,
    help="Loss scaling to improve fp16 numeric stability. Only used when fp16 set to True.\n"
    "0 (default value): dynamic loss scaling.\n"
    "Positive power of 2: static loss scaling value.\n",
)
parser.add_argument(
    "--num_workers",
    type=int,
    default=16,
    help="Number of workers in the dataloader.",
)
parser.add_argument(
    "--save_name", default="", type=str, help="save name for training."
)
parser.add_argument(
    "--use_chunk",
    default=0,
    type=float,
    help="whether use chunck for parallel training.",
)
parser.add_argument(
    "--batch_size", default=30, type=int, help="what is the batch size?"
)
parser.add_argument(
    "--tasks", default="", type=str, help="1-2-3... training task separate by -"
)
parser.add_argument(
    "--in_memory",
    default=False,
    type=bool,
    help="whether use chunck for parallel training.",
)
parser.add_argument(
    "--baseline", action="store_true", help="whether use single stream baseline."
)
parser.add_argument("--split", default="", type=str, help="which split to use.")
parser.add_argument(
    "--dynamic_attention",
    action="store_true",
    help="whether use dynamic attention.",
)
parser.add_argument(
    "--clean_train_sets",
    default=True,
    type=bool,
    help="whether clean train sets for multitask data.",
)
parser.add_argument(
    "--visual_target",
    default=0,
    type=int,
    help="which target to use for visual branch. \
    0: soft label, \
    1: regress the feature, \
    2: NCE loss.",
)
parser.add_argument(
    "--task_specific_tokens",
    action="store_true",
    help="whether to use task specific tokens for the multi-task learning.",
)

_StoreTrueAction(option_strings=['--task_specific_tokens'], dest='task_specific_tokens', nargs=0, const=True, default=False, type=None, choices=None, help='whether to use task specific tokens for the multi-task learning.', metavar=None)

## load the textual input

In [4]:
args = parser.parse_args(['--bert_model', 'bert-base-uncased',
                          '--from_pretrained', 'save/ME_bert_base_6layer_6conect-finetune_from_multi_task_model-task_19-all_train-NLVR2/pytorch_model_11.bin',
                          '--config_file', 'config/bert_base_6layer_6conect.json',
                          '--tasks', '19',
                          '--split', 'test',
                          '--save_name', 'task-19-end-to-end',
                          '--task_specific_tokens',
                          '--batch_size', '128'])

In [5]:
with open("vilbert_tasks.yml", "r") as f:
    task_cfg = edict(yaml.safe_load(f))

random.seed(args.seed)
np.random.seed(args.seed)
torch.manual_seed(args.seed)

if args.baseline:
    from pytorch_transformers.modeling_bert import BertConfig
    from vilbert.basebert import BaseBertForVLTasks
else:
    from vilbert.vilbert import BertConfig
    from vilbert.vilbert import VILBertForVLTasks

task_names = []
for i, task_id in enumerate(args.tasks.split("-")):
    task = "TASK" + task_id
    name = task_cfg[task]["name"]
    task_names.append(name)

# timeStamp = '-'.join(task_names) + '_' + args.config_file.split('/')[1].split('.')[0]
timeStamp = args.from_pretrained.split("/")[-1] + "-" + args.save_name
savePath = os.path.join(args.output_dir, timeStamp)
config = BertConfig.from_json_file(args.config_file)

if args.task_specific_tokens:
    config.task_specific_tokens = True

if args.local_rank == -1 or args.no_cuda:
    device = torch.device(
        "cuda" if torch.cuda.is_available() and not args.no_cuda else "cpu"
    )
    n_gpu = torch.cuda.device_count()
else:
    torch.cuda.set_device(args.local_rank)
    device = torch.device("cuda", args.local_rank)
    n_gpu = 1
    # Initializes the distributed backend which will take care of sychronizing nodes/GPUs
    torch.distributed.init_process_group(backend="nccl")

logger.info(
    "device: {} n_gpu: {}, distributed training: {}, 16-bits training: {}".format(
        device, n_gpu, bool(args.local_rank != -1), args.fp16
    )
)

default_gpu = False
if dist.is_available() and args.local_rank != -1:
    rank = dist.get_rank()
    if rank == 0:
        default_gpu = True
else:
    default_gpu = True

if default_gpu and not os.path.exists(savePath):
    os.makedirs(savePath)

task_batch_size, task_num_iters, task_ids, task_datasets_val, task_dataloader_val = LoadDatasetEval(
    args, task_cfg, args.tasks.split("-")
)

tbLogger = utils.tbLogger(
    timeStamp,
    savePath,
    task_names,
    task_ids,
    task_num_iters,
    1,
    save_logger=False,
    txt_name="eval.txt",
)
# num_labels = max([dataset.num_labels for dataset in task_datasets_val.values()])

if args.dynamic_attention:
    config.dynamic_attention = True
if "roberta" in args.bert_model:
    config.model = "roberta"

if args.visual_target == 0:
    config.v_target_size = 1601
    config.visual_target = args.visual_target
else:
    config.v_target_size = 2048
    config.visual_target = args.visual_target

if args.task_specific_tokens:
    config.task_specific_tokens = True

08/28/2020 12:13:11 - INFO - __main__ -   device: cuda n_gpu: 1, distributed training: False, 16-bits training: False
08/28/2020 12:13:12 - INFO - pytorch_transformers.tokenization_utils -   loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/aloui/.cache/torch/pytorch_transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
08/28/2020 12:13:12 - INFO - vilbert.task_utils -   Loading ME Dataset with batch size 128
08/28/2020 12:13:12 - INFO - vilbert.datasets.me_dataset -   Loading from datasets/ME/cache/ME_test_23_cleaned.pkl
08/28/2020 12:13:12 - INFO - vilbert.utils -   logging file at: pytorch_model_10.bin-task-19-end-to-end


In [6]:
task_batch_size, task_num_iters, task_ids, task_datasets_val, task_dataloader_val

({'TASK19': 128},
 {'TASK19': 16},
 ['TASK19'],
 {'TASK19': <vilbert.datasets.me_dataset.MERegressionDataset at 0x7f2090ad8940>},
 {'TASK19': <torch.utils.data.dataloader.DataLoader at 0x7f2096ed3160>})

In [7]:
len(task_datasets_val['TASK19']), len(task_dataloader_val['TASK19'])

(2000, 16)

## load the pretrained model

In [8]:
num_labels = 2

if args.baseline:
    model = BaseBertForVLTasks.from_pretrained(
        args.from_pretrained,
        config=config,
        num_labels=num_labels,
        default_gpu=default_gpu,
    )
else:
    model = VILBertForVLTasks.from_pretrained(
        args.from_pretrained,
        config=config,
        num_labels=num_labels,
        default_gpu=default_gpu,
    )

task_losses = LoadLosses(args, task_cfg, args.tasks.split("-"))
model.to(device)
if args.local_rank != -1:
    try:
        from apex.parallel import DistributedDataParallel as DDP
    except ImportError:
        raise ImportError(
            "Please install apex from https://www.github.com/nvidia/apex to use distributed and fp16 training."
        )
    model = DDP(model, delay_allreduce=True)

elif n_gpu > 1:
    model = nn.DataParallel(model)

08/28/2020 12:13:14 - INFO - vilbert.utils -   loading weights file save/ME_bert_base_6layer_6conect-finetune_from_multi_task_model-task_19-all_train-NLVR2/pytorch_model_10.bin


## Propagate Training Split

In [9]:
print("***** Running evaluation *****")
print("  Num Iters: ", task_num_iters)
print("  Batch size: ", task_batch_size)

targets_list, preds_list = list(), list()

model.eval()
# when run evaluate, we run each task sequentially.
for task_id in task_ids:
    results = []
    others = []
    for i, batch in enumerate(task_dataloader_val[task_id]):
        loss, score, batch_size, results, others, vil_score_prediction = EvaluatingModel(
            args,
            task_cfg,
            device,
            task_id,
            batch,
            model,
            task_dataloader_val,
            task_losses,
            results,
            others,
        )
        targets_list.append(batch[4]) # target
        preds_list.append(vil_score_prediction)
        
        tbLogger.step_val(0, float(loss), float(score), task_id, batch_size, "val")
        sys.stdout.write("%d/%d\r" % (i, len(task_dataloader_val[task_id])))
        sys.stdout.flush()
    # save the result or evaluate the result.
    ave_score = tbLogger.showLossVal(task_id)

    if args.split:
        json_path = os.path.join(savePath, args.split)
    else:
        json_path = os.path.join(savePath, task_cfg[task_id]["val_split"])

    json.dump(results, open(json_path + "_result.json", "w"))
    json.dump(others, open(json_path + "_others.json", "w"))

***** Running evaluation *****
  Num Iters:  {'TASK19': 16}
  Batch size:  {'TASK19': 128}


08/28/2020 12:13:30 - INFO - vilbert.utils -   Eval task TASK19 on iteration 0 
08/28/2020 12:13:30 - INFO - vilbert.utils -   Validation [ME]: loss 0.026 score 0.381 


In [10]:
len(targets_list), len(preds_list)

(16, 16)

In [11]:
targets = torch.cat(targets_list, 0)
preds = torch.cat(preds_list, 0)

In [12]:
r, _ = spearmanr(
    targets.cpu().detach().numpy()[:,0], 
    preds.cpu().detach().numpy()[:,0], 
    axis=0
)
r

0.4771996524236738

In [13]:
r, _ = spearmanr(
    targets.cpu().detach().numpy()[:,1], 
    preds.cpu().detach().numpy()[:,1], 
    axis=0
)
r

0.22844158655840108